# A simple BCH error correction tutorial (python)

### Import libraries

In [3]:
import bchlib
import numpy as np
import random

### Utility functions that convert bitstring <-> btyearray

In [4]:
def bitstring_to_bytearray(data):
    is_first = True
    
    while len(data)%8!=0:
        data = np.append(data,0)
    while len(data) >=8:
        substring = ''
        for i in range(8):
            substring += str(data[i])
        
        if is_first:
            btye_array = bytearray([int(substring,2)])
            is_first= False
        
        else:
            btye_array += bytearray([int(substring,2)])
        data = data[8:]
        
    if (list(data)):
        substring = ''
        for i in range(len(data)):
            substring += str(data[i])
                    
    return btye_array

In [5]:
def bytearray_to_bits(x):
    result = []
    for i in x:
        bits = bin(i)[2:]
        bits = '00000000'[len(bits):] + bits
        result.extend([int(b) for b in bits])

    return result

## Create a BCH module

In [6]:
BCH_POLYNOMIAL = 8219
BCH_BITS = 3
bch = bchlib.BCH(BCH_POLYNOMIAL,BCH_BITS)

### Create Sample Binary String

Generate a sample bit-string by using Numpy

In [7]:
length = 30
data = np.random.choice([0,1], length)
print(data)

[0 1 0 1 1 0 1 1 1 1 0 1 1 1 0 1 1 0 0 1 1 1 0 1 0 1 0 1 1 1]


To encode ecc code, we should convert the bit-string to btyearray 

In [8]:
barray = bitstring_to_bytearray(data)
print(barray)

bytearray(b'[\xdd\x9d\\')


To show the opposite side, we convert bit-string to bytearray

In [9]:
bitstring = (bytearray_to_bits(barray))
print(bitstring)

[0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0]


## BCH encode.
We leverage the BCH module for generate ECC (Error Correction Code).

When we transmit the data, we generate the packet (data + ecc) for error correction.

In [10]:
ecc = bch.encode(barray)

In [11]:
packet = barray + ecc

In [12]:
print(packet)

bytearray(b'[\xdd\x9d\\\x1d\x8e\x9d-\xd0')


## Simple packet error
To test error correction, we artifically make a simple noise situation that some bits are changed to 1.

In [13]:
error_occur = bytearray_to_bits(packet)
for i in range(5):
    r_number = random.randrange(0,len(error_occur))
    error_occur[r_number] = 1
    print(r_number,'th is changed to 1')
    print(error_occur)
transmitted = bitstring_to_bytearray(error_occur)

64 th is changed to 1
[0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0]
42 th is changed to 1
[0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0]
20 th is changed to 1
[0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0]
20 th is changed to 1
[0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0]
40 th is changed to 1
[0, 1, 0, 1, 1, 0, 1, 

# BCH Decode
When the data is transmitted, we can correct the error by decoding. 

To decode, we use BCH module. The first element is data part, the second is ecc part. 

We can easily split these parts by silicing. 

In [14]:
decoded = bch.decode(transmitted[:-len(ecc)],ecc)

In [15]:
print(decoded[1])

bytearray(b'[\xdd\x9d\\')


In [16]:
print(bytearray_to_bits(decoded[1])[:-2])

[0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1]


In [17]:
print(data)

[0 1 0 1 1 0 1 1 1 1 0 1 1 1 0 1 1 0 0 1 1 1 0 1 0 1 0 1 1 1]


## Result

In [18]:
xor = 0
for idx in range(len(data)):
    if data[idx] != bytearray_to_bits(decoded[1])[:-2][idx]:
        xor+=1
print(xor)

0
